In [1]:
import torch
import torch.optim
import numpy as np

from nni.nas.space import model_context
from nni.nas.strategy import DARTS as DartsStrategy
from nni.nas.experiment import NasExperiment
from dataset.classification import fetch_data
from evaluators.classification import ClassificationEvaluator
from models.mlp import MLP

In [2]:
np.random.seed(0)
torch.random.manual_seed(0)

# Fetch dataset loaders

In [3]:
task_config, loaders = fetch_data('ja', batch_size=256, num_workers=4)

In [4]:
for split_name, loader in loaders.items():
    print(split_name, 'dataset size:', len(loader.dataset))

print('num_classes:', task_config.out_features)

train dataset size: 53588
val dataset size: 13398
test dataset size: 16747
num_classes: 4


# Training

In [5]:
evaluator = ClassificationEvaluator(
    learning_rate=3e-4,
    weight_decay=1e-5,
    optimizer=torch.optim.AdamW,
    train_dataloaders=loaders['train'],
    val_dataloaders=loaders['val'],
    num_classes=task_config.out_features,
    max_epochs=100,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
strategy = DartsStrategy()
model_space = MLP(d_in=task_config.in_features, d_out=task_config.out_features, dropout=0.1)
experiment = NasExperiment(model_space, evaluator, strategy)
experiment.run()

[2024-05-15 15:00:49] Config is not provided. Will try to infer.
[2024-05-15 15:00:49] Strategy is found to be a one-shot strategy. Setting execution engine to "sequential" and format to "raw".
[2024-05-15 15:00:49] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-15 15:00:49] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-15 15:00:49] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-15 15:00:50] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-15 15:00:50] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-15 15:00:50] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-15 15:00:50] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-15 15:00:50] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-15 15

You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                 | Params
---------------------------------------------------------
0 | training_module | ClassificationModule | 2.7 M 
---------------------------------------------------------
2.7 M     Trainable params
0         Non-trainable params
2.7 M     Total params
10.627    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


[2024-05-15 15:14:11] Waiting for models submitted to engine to finish...
[2024-05-15 15:14:11] Experiment is completed.
[2024-05-15 15:14:11] WARNING: `training_service` will be ignored for sequential execution engine.


True

# Train final model

In [7]:
exported_arch = experiment.export_top_models(formatter='dict')[0]
print(exported_arch)

{'MLP/d_block': 32, 'MLP/in_act': 0, 'MLP/n_blocks': 1, 'MLP/blocks_act': 0}


In [8]:
with model_context(exported_arch):
    final_model = MLP(d_in=task_config.in_features, d_out=task_config.out_features, dropout=0.1)

In [9]:
evaluator = ClassificationEvaluator(
    learning_rate=3e-4,
    weight_decay=1e-5,
    optimizer=torch.optim.AdamW,
    train_dataloaders=loaders['train'],
    val_dataloaders=loaders['val'],
    num_classes=task_config.out_features,
    max_epochs=150,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
evaluator.fit(final_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | metrics   | ModuleDict       | 0     
2 | _model    | MLP              | 2.9 K 
-----------------------------------------------
2.9 K     Trainable params
0         Non-trainable params
2.9 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[2024-05-15 15:14:52] Intermediate result: 0.5675473809242249  (Index 0)


...

[2024-05-15 15:17:51] Intermediate result: 0.7100313305854797  (Index 148)


Validation: |          | 0/? [00:00<?, ?it/s]

[2024-05-15 15:17:52] Intermediate result: 0.7093595862388611  (Index 149)


`Trainer.fit` stopped: `max_epochs=150` reached.


[2024-05-15 15:17:53] Final result: 0.7093595862388611


# Evaluate final model

In [11]:
test_evaluator = ClassificationEvaluator(
    learning_rate=3e-4,
    weight_decay=1e-5,
    optimizer=torch.optim.AdamW,
    val_dataloaders=loaders['test'],
    num_classes=task_config.out_features,
    max_epochs=100,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
test_evaluator.evaluate(final_model)

[2024-05-15 15:18:22] Only validation dataloaders are available. Skip to validation.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

[2024-05-15 15:18:23] Intermediate result: 0.7114109992980957  (Index 150)
[2024-05-15 15:18:23] Final result: 0.7114109992980957
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.7114109992980957
        val_loss            0.6988751888275146
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.6988751888275146, 'val_acc': 0.7114109992980957}]